In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from random import choices, sample
from gensim.models import Word2Vec
import gc
pd.set_option('display.float_format',lambda x : '%.2f' % x)
from math import log10
from collections import Counter

In [2]:
PART = '2'

In [3]:
with open('../Taobao_v3/dict_knn_table/knn_table_'+ PART +'.pkl', 'rb') as file:
    knn_table = pickle.load(file)

In [4]:
with open('./freq_dict.pkl', 'rb') as file:
    target_log_dict, next_log_dict = pickle.load(file)

In [5]:
model_w2v = Word2Vec.load("../Taobao_v3/iid_1days.model")

In [6]:
def get_embedding(target_iid, next_iid):
    if target_iid in target_log_dict:
        tar_freq = target_log_dict[int(target_iid)]
    else:
        tar_freq = 0
    if next_iid in next_log_dict:
        next_freq = next_log_dict[int(next_iid)]
    else:
        next_freq = 0
    freq = np.append([tar_freq], [next_freq])
    emb_pair = np.append(model_w2v.wv[target_iid], model_w2v.wv[next_iid])
    embedding = np.append(freq, emb_pair)
    return embedding

In [7]:
from keras import models
model = models.load_model('./saved_model/Ali_switch_0602_G.h5')

Using TensorFlow backend.


In [8]:
pred_dict = {}

In [9]:
for target_iid in tqdm(list(knn_table.keys())):
    topn, cos = zip(*knn_table[target_iid])
    pred_list = []
    for next_iid in topn:
        pred_list.append(np.argmax(model.predict(get_embedding(target_iid, next_iid).reshape(1, -1))[0]))
    pred_dict[int(target_iid)] = Counter(pred_list).most_common()[0][0]

 29%|██▉       | 111280/381339 [1:30:39<3:41:24, 20.33it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 35%|███▌      | 134938/381339 [1:49:56<3:20:49, 20.45it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 42%|████▏     | 158470/381339 [2:09:07<3:01:33, 20.46it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config varia

In [10]:
with open('./pred_dict/pred_dict_' + PART + '.pkl', 'wb') as file:
    pickle.dump(pred_dict, file)

In [11]:
len(pred_dict)

381339